# Model Training Notebook
This notebook uses training dataset to perform model training. It uses SageMaker Autopilot to train multiple models and select the best
perfoming model. The problem is defined to be a `binary classification` problem of accepting or rejecting a pacs.008 message.

![SageMaker Autopilot](images/iso20022-prototype-AutoML.png)

In [1]:
import warnings
warnings.filterwarnings("ignore")

## Environment Setup

In [ ]:
import os
import boto3
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role

sm = boto3.Session().client('sagemaker')
sess = sagemaker.Session()
region = boto3.session.Session().region_name

role = get_execution_role()
print ("Notebook is running with assumed role {}".format (role))
print("Working with AWS services in the {} region".format(region))

### Provide S3 Bucket Name

In [ ]:
# Working directory for the notebook
WORKDIR = os.getcwd()
BASENAME = os.path.dirname(WORKDIR)

# Store all prototype assets in this bucket
s3_bucket_name = 'iso20022-prototype-t3'
s3_bucket_uri = 's3://' + s3_bucket_name

# Prefix for all files in this prototype
prefix = 'iso20022'

pacs008_prefix = prefix + '/pacs008'
raw_data_prefix = pacs008_prefix + '/raw-data'
labeled_data_prefix = pacs008_prefix + '/labeled-data'
training_data_prefix = pacs008_prefix + '/automl/training-data'
training_headers_prefix = pacs008_prefix + '/automl/training-headers'
test_data_prefix = pacs008_prefix + '/automl/test-data'
training_job_output_prefix = pacs008_prefix + '/training-output'

print(f"Training data will be uploaded to {s3_bucket_uri + '/' + training_data_prefix}")
print(f"Test data will be uploaded to {s3_bucket_uri + '/' + test_data_prefix}")
print(f"Training job output will be stored in {s3_bucket_uri + '/' + training_job_output_prefix}")

In [ ]:
labeled_data_location = s3_bucket_uri + '/' + labeled_data_prefix
training_data_location = s3_bucket_uri + '/' + training_data_prefix
test_data_location = s3_bucket_uri + '/' + test_data_prefix
print(f"Raw labeled data location = {labeled_data_location}")
print(f"Training data location = {training_data_location}")
print(f"Test data location = {test_data_location}")

# Split Labeled Dataset to Training and Test Datasets

In [ ]:
# Download labeled raw dataset from S3
s3 = boto3.client('s3')
s3.download_file(s3_bucket_name, labeled_data_prefix + '/labeled_data.csv', 'labeled_data.csv')
df = pd.read_csv('labeled_data.csv')
df.shape

In [ ]:
df.head()

### Select Features

In [ ]:
# Training features
fts=[
 'y_target',   
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_Dbtr_PstlAdr_Ctry',  
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_Cdtr_PstlAdr_Ctry', 
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_RgltryRptg_DbtCdtRptgInd', 
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_RgltryRptg_Authrty_Ctry', 
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_RgltryRptg_Dtls_Cd',
 'Document_FIToFICstmrCdtTrf_CdtTrfTxInf_InstrForNxtAgt_InstrInf',
]

# New data frame with selected features
selected_df = df[fts]
    
print(f"selected_df shape: {selected_df.shape}")    
selected_df.head()

### Split into Training and Test Datasets

In [ ]:
from sklearn.model_selection import train_test_split
# Split raw labeled data to training and test datasets
print('Spliting processed dataset into training and test datasets...')

X_train, X_test, y_train, y_test = train_test_split(df, df['y_target'], test_size=0.2, random_state=20, shuffle=True)
#X_train, X_test, y_train, y_test = train_test_split(selected_df, selected_df['y_target'], test_size=0.2, random_state=20, shuffle=True)

print(f"X_train shape: {X_train.shape}")    
print(f"X_test shape: {X_test.shape}") 
print(f"y_train shape: {y_train.shape}") 
print(f"y_test shape: {y_test.shape}") 

train_data_output_path = WORKDIR + '/train_data.csv'

test_data_output_path =  WORKDIR + '/test_data.csv'

print("Saving training data with headers to {}".format(train_data_output_path))
X_train.to_csv(train_data_output_path, index=False)

print('Saving test data with headers to {}'.format(test_data_output_path))
X_test.to_csv(test_data_output_path, index=False)

s3.upload_file(train_data_output_path, s3_bucket_name, training_data_prefix + '/train_data.csv')
s3.upload_file(test_data_output_path, s3_bucket_name, test_data_prefix + '/test_data.csv')

print(f'Uploaded train data with headers to {training_data_location}')
print(f'Uploaded test data with headers to {test_data_location}')

print("Pre-processing Complete.")

# Create a Model Using SageMaker Autopilot

## Setting the Sagemaker Autopilot Job

In [ ]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': training_data_location ## Where the training data is stored 
        }
      },
      'TargetAttributeName': 'y_target' ## Name of the target value 
    }
  ]

output_data_config = {
    'S3OutputPath': s3_bucket_uri + '/' + training_job_output_prefix ## Where to store the model performance
  }

autoMLJobConfig={
        'CompletionCriteria': {
            'MaxCandidates': 5 ## Number of models you want to try 
        }
}

autoMLJobObjective = {
      "MetricName": "Accuracy" ## Metric we want to use to evaluate the model
}

In [ ]:
print(input_data_config)

## Start SageMaker Autopilot job

In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%Y-%m-%d-%H-%M', gmtime())

auto_ml_job_name = 'pacs008-automl-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

#auto_ml_job_name = 'automl-iso20022-2021-11-25-16-13' # selected_df
#auto_ml_job_name = '' #full labeled df

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=autoMLJobConfig,
                      AutoMLJobObjective=autoMLJobObjective,
                      ProblemType="BinaryClassification", ## Here we specify what type of problem we have
                      RoleArn=role)

### Monitor Training Job

In [ ]:
from time import sleep
print ('JobStatus - Secondary Status')
print('------------------------------')

#auto_ml_job_name = 'automl-iso20022-2021-11-09-22-42'
print('AutoMLJobName: ' + auto_ml_job_name)
describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

### Autopilot Training Results

In [ ]:
import pprint

best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
pprint.pprint(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
best_candidate['InferenceContainers'][1]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label,probability'})
pprint.pprint(best_candidate['InferenceContainers'][1])

best_candidate['InferenceContainers'][2]['Environment'].update({'SAGEMAKER_INFERENCE_INPUT': 'predicted_label,probability'})
best_candidate['InferenceContainers'][2]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label,probability'})
pprint.pprint(best_candidate['InferenceContainers'][2])

pprint.pprint(best_candidate)

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

### Autopilot Generated Candidate Model Training Notebook

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']

### Autopilot Generated Data Exploration Notebook

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']

## Store Training Job Name for use during Model Deployment
This name is used by model deployment notebook that deploys a SageMaker Inference Endpoint i.e. uses SageMaker hosting services to deploy the model and expose an inference endpoint for users to use the model to make predictions.

In [ ]:
training_job_name = candidates[0]['CandidateName']
%store training_job_name

In [ ]:
%store

In [ ]:
# timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
#model_name = 'pacs008-automl-' + timestamp_suffix

model_name = auto_ml_job_name
model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

In [ ]:
%store model_name

In [ ]:
%store